# 5.1 Region-based techniques for image segmentation

Image segmentation is the process of **assigning a label to every pixel in an image** such that pixels with the same label share certain characteristics. As a consequence, it produces regions whose pixels have similar properties, e.h. intensity, color, texture, location in the image. The result of image segmentation could be:
- a set of segments that collectively cover the entire image (e.g. thresholding), 
- or a set of contours extracted from the image (e.g. edge detection).

<img src="./images/image-segmentation-example.png" width="550"/>$\\[5pt]$

Conceptually, two traditional approaches to image segmentation exist:
- **Top-down segmentation**, which considers that pixels from the same object in the scene should be in the same segmented region.
- **Bottom-up segmentation**, which establishes that similar pixels in the image must be in the same segmented region. $\\[5pt]$

<center><img src="./images/bottom-up_top-down-segmentation.png" width="500"/>Image adopted from <a href="myfootnote1">[1]</a></center>$\\[5pt]$

We put the spotlight here on bottom-up segmentation approaches. Methods following such approach can be grouped into:
- **Contour-based techniques**, which attemp to identify the image regions by detecting their contours.
- **Region-based techniques** that group together pixels that are similar. 

In this book, we will cover both families of techniques, starting with two popular region-based methods: 

- K-means (section 5.1.1)
- Expectation-Maximization (EM, section 5.1.2)

## Problem context - Color quantization

<img src="./images/color-quantization.jpg" width="800"/>$\\[5pt]$

Color quantization is the process of reducing the number of distinct colors in an image while preserving its color appearance as much as possible. It has many applications, like image compression (e.g. GIFs) or [content-based image retrieval](https://en.wikipedia.org/wiki/Content-based_image_retrieval). 

Image segmentation techniques can be used to achieve color quantization, let's see how it works!

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib
import scipy.stats as stats
from ipywidgets import interact, fixed, widgets
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
images_path = './images/'

import sys
sys.path.append("..")
from utils.PlotEllipse import PlotEllipse

## 5.1.1 K-Means

As commented, region-based techniques try to group together pixels that are similar. Such problem is often called the *clustering problem*. Different attributes can be used to decide if two pixels are similar or not: intensity, texture, color, pixel location, etc.

The **k-means algorithm** is a region-based technique that, given a set of elements, makes $K$ clusters out of them. Thereby, it is a perfect technique for addressing color quantization, since our goal is to reduce the color palette of an image to a fixed number of colors $K$.

But, **how k-means algorithm works** in a color domain, where each pixel is represented in a n-dimensional space? (e.g. grayscale images define a 1D space, while RGB images a 3D space):

1. Pick the number $K$, that is, the number of clusters in which image will be segmented (e.g. number of colors).
2. Place $K$ centroids in the color space (e.g. randomly), these are the centers of the regions. 
3. Each pixel is assigned to the cluster with the closest centroid, hence creating new clusters. 

<img src="./images/kmeans-step-1.png" width="400"/><center><i>Example in a 2D space (e.g. YCbCr color space) with 3 clusters. Each point is assigned to its closer centroid</i></center>$\\[5pt]$

4. Compute the new means of the clusters. 

<img src="./images/kmeans-step-2.png" width="400" align="center"/><center><i>Example of how the centroids evolve over time</i></center>$\\[5pt]$

5. Repeat steps 3 and 4 until convergence, that is, some previously defined criteria is fulfilled.$\\[10pt]$

<img src="./images/kmeans-step-3.png" width="400" align="center"/><center><i>Final segmentation result</i></center>$\\[5pt]$

The procedure is the same independently of the number of dimensions in the workspace. 

This technique presents a number of pros and cons:

- **Pros:**
  - It's simple.
  - Convergence to a local minima is guaranteed (but no guarantee to reach the global minima).
- **Cons:**
  - High usage of memory.
  - The K must be fixed.
  - Sensible to the selection of the initialization (initial position of centroids).
  - Sensible to outliers.
  - Circular clusters in the feature space are assumed (because of the usage of the Euclidean distance)

### K-means toy example

Luckily for us, OpenCV defines a method that perform k-means: [`cv2.kmeans()`](https://docs.opencv.org/2.4/modules/core/doc/clustering.html). Let's take a look at a toy 1D k-means example in order to get familiar with it. The following function, `binarize_kmeans()`, binarizes an input `image` by executing the K-means algorithm, where the `it` sets its maximum number of iterations (it represents stopping/convergence criteria). 

In [ ]:
def binarize_kmeans(image,it):
    """ Binarize an image using k-means.   

        Args:
            image: Input image
            it: K-means iteration
    """
    
    
    # Set random seed for centroids 
    cv2.setRNGSeed(124)
    
    # Flatten image
    flattened_img = image.reshape((-1,1))
    flattened_img = np.float32(flattened_img)
    
    #Set epsilon
    epsilon = 0.2
    
    # Estabish stopping criteria (either `it` iterations or moving less than `epsilon`)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, it, epsilon)
    
    # Set K parameter (2 for thresholding)
    K = 2
    
    # Call kmeans using random initial position for centroids
    _,label,center=cv2.kmeans(flattened_img,K,None,criteria,it,cv2.KMEANS_RANDOM_CENTERS)
    
    # Colour resultant labels
    center = np.uint8(center)
    flattened_img = center[label.flatten()]
    
    # Reshape vector image to original shape
    binarized = flattened_img.reshape((image.shape))
    
    # Show resultant image
    plt.subplot(2,1,1)
    plt.title("Original image")
    plt.imshow(binarized, cmap='gray',vmin=0,vmax=255)
    
    # Show how original histogram have been segmented
    plt.subplot(2,1,2)
    plt.title("Segmented histogram")
    plt.hist([image[binarized==center[0]].ravel(), image[binarized==center[1]].ravel()],256,[0,256], color=["black","gray"],stacked="true")


Note that the stopping criteria can be either if a maximum number of iterations is reached, or if the centroid moved less than a certain **epsion** value in a iteration.

As you can see, [`cv2.kmeans()`](https://docs.opencv.org/2.4/modules/core/doc/clustering.html) returns two relevant arguments:

- label: Integer array that stores the cluster index for every sample.
- center: Matrix containing the cluster centroids (each row represents a different centroid)

It is provided below an interactive code so you can play with `cv2.kmeans()` by calling it with different `it`values. After trying it **you are asked to explain** what `cv2.kmeans()` is doing in each iteration.

*As you can see, if k=2 in a grayscale image, it is a binarization method that doesn't need to fix a manual threshold. We could have used it, for example, when dealing with the plate recognition problem!*

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)

image = cv2.imread(images_path + 'plate.jpg',0)

interact(binarize_kmeans, image=fixed(image),it=(2,5,1))

Notice that for 1D spaces and not high-resolution images k-means is very fast! (it only needs a few iterations to converge). What happens if k-means is applied with color images (3D space) in order to get color quantization?  

Now that you know how k-means works, you can experimentally answer such question!  

###  **<span style="color:green"><b><i>ASSIGNMENT 1: Playing with K-means</i></b></span>**

Write an script that:
- applies k-means to `malaga.png` with different values for $K$: $K=4$, $K=8$ and $K=16$,
- shows, in a $2\times2$ subplot, the 3 resulting images along with the input one.

<font color='blue'>**Expected output:**  </font>

<img src="images/exercise1.png" width="800"/>

In [ ]:
# Assignment 1
matplotlib.rcParams['figure.figsize'] = (15.0, 12.0)

# Read RGB image


# Flatten image

# Set criteria


# Apply k-means


# Colour resultant labels


# Reshape to original shape


# Show original image


# Show k=4


# Show k=8


# Show k=16


### <font color="blue"><b><i>Thinking about it (1)</i></b></font>

Now, **answer the following questions**:

- What number of maximum iterations did you use? Why?
  
    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- How could it be possible to compress these images? *Note: consider that a pixel needs 3 bytes to be represented, 8 bits per band.*
  
    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>   

###  **<span style="color:green"><b><i>EXTRA ASSIGNMENT: Analyzing execution times</i></b></span>**

In this exercise you are asked to compare the execution time of K-means in a grayscale image, with K-means in a RGB image. Use the image `malaga.png` for this task, and use the same number of clusters and criteria for both, the grayscale and the RGB images.

*Tip: [how to measure execution time in Python](https://stackoverflow.com/questions/14452145/how-to-measure-time-taken-between-lines-of-code-in-python)*

In [ ]:
import time

print("Measuring the execution time needed for ...")


# Read images


# Set criteria


# Start timer

# Flatten image


# Apply k-means


# Stop timer

# Start timer

# Flatten image


# Apply k-means


# Stop timer

## 5.1.2 Expectation-Maximization (EM)

**Expectation-Maximization (EM)** is the generalization of the K-means algorithm, where each cluster is represented by a Gaussian distribution, parametrized by a mean and a covariance matrix, instead of just a centroid. It's a *soft clustering* since it doesn't give *hard* decisions where a pixel belongs or not to a cluster, but the probability of that pixel belonging to each cluster $C_j$, that is, $p(x|C_j) \sim N(\mu_j,\Sigma_j)$. This implies that at each algorithm iteration not just the mean of each cluster is refined (as in K-means), but also their covariance matrices.

Before going into detail on the theory behind EM, it is worth seeing how it performs in the car plate problem. OpenCV provides a class implementing the needed functionality for applying EM segmentation to an image, called [cv2.ml.EM](https://docs.opencv.org/3.4/d1/dfb/classcv_1_1ml_1_1EM.html). All methods and parameters are fully detailed in the documentation, so it is a good idea to take a look at it.

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
cv2.setRNGSeed(5)

# Define parameters
n_clusters = 2 
covariance_type = 0 # 0: covariance matrix spherical. 1: covariance matrix diagonal. 2: covariance matrix generic
n_iter = 10
epsilon = 0.2

# Create EM empty object
em = cv2.ml.EM_create()

# Set parameters
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, n_iter, epsilon)
em.setClustersNumber(n_clusters)
em.setCovarianceMatrixType(covariance_type)
em.setTermCriteria(criteria)

# Read grayscale image
image = cv2.imread(images_path + "plate.jpg",0)

# Flatten image
flattened_img = image.reshape((-1,1))
flattened_img = np.float32(flattened_img)

# Apply EM
_, _, labels, _ = em.trainEM(flattened_img)

# Reshape labels to image size (binarization)
binarized = labels.reshape((image.shape))

# Show original image
plt.subplot(2,1,1)
plt.title("Binarized image")
plt.imshow(binarized, cmap="gray")

# --------------- Gaussian visualization ---------------

plt.subplot(2,1,2)
plt.title("Probabilities of the clusters")

# Get means and covs (for grayscale 1D both)
means = em.getMeans()
covs = em.getCovs()

# Get standard deviation as numPy array
sigmas = np.sqrt(covs)
sigmas = sigmas[:,0,0]

# Cast list to numPy array
means = np.array(means)[:,0]

# Plot Gaussians
x = np.linspace(0, 256, 100)
plt.plot(x, stats.norm.pdf(x, loc = means[0], scale = sigmas[0]))
plt.plot(x, stats.norm.pdf(x, loc = means[1], scale = sigmas[1]))
plt.legend(['Black Region', 'White Region'])

plt.show()

As you can see, although in OpenCV k-means is implemented as a method and EM as a class, they operate in a similar way. In the example above, we are segmenting a car plate into two clusters, and **each cluster is defined by a Gaussian distribution** (a Gaussian distribution for the black region, and another one for the white region). This is the basis of EM, **but how it works**?  

EM is an iterative algorithm that is divided into two main steps:

- First of all, it **initializes the mean and covariance matrix of each of the $K$ clusters**. Typically, it picks at random ($\mu_j$,$\Sigma_j$) and $P(C_j)$ (prior probability) for each cluster $j$.
- Then, it keeps iterating doing Expectation-Maximization steps until some stopping criteria is satisfied (e.g. when no change occurs in a complete iteration):$\\[1pt]$
    1. **Expectation step:** calcule the probabilities of every point belonging to each cluster, that is $p(C_j|x_i), \forall i \in data$:
    $$P(C_j|x_i)=\frac{p(x_i|C_j)p(C_j)}{p(x_i)}=\frac{p(x_i|C_j)p(C_j)}{\sum_i P(x_i|C_j)p(C_j)}$$
    assign $x_i$ to the cluster $C_j$ with the highest probability $P(C_j|x_i)$.$\\[10pt]$
    2. **Maximization step:** re-estimate the cluster parameters (($\mu_j$,$\Sigma_j$) and $p(C_j)$) for each cluster $j$ knowing the expectation step results, which is also called *Maximum Likelihood Estimate* (MLE):
    $$\mu_j=\frac{\sum_i p(C_j|x_i)x_i}{\sum_i p(C_j|x_i)}\\
    \sum_j = \frac{\sum_i p(C_j|x_i)(x_i-\mu_j)(x_i-\mu_j)^T}{\sum_i p(C_j|x_i)}\\
    p(C_j)=\sum_i p(C_j|x_i)p(x_i)=\frac{\sum_i p(C_j|x_i)}{N}$$
    *Note that if no other information is available, the priors are considered equally probable.*
    
<img src="./images/em.gif" width="400" align="center"><center>*Example of an execution of the EM algorithm with two clusters, with details about the evolution of their associated Gaussian distributions.*</center>$\\[5pt]$


Doesn't it remind you to the K-means algorithm? **What is the difference between them?** 

The main difference is that K-means employs the **euclidian distance** to measure how near is a point to a cluster. In EM we use a distance in which **each dimension is weighted** by the **covariance matrix** of each cluster, which is also called **Mahalanobis distance**. Furthermore, for k-means a point of data **belongs or not to** a cluster, in EM a point of data have a higher or lower **probability** to belong to a cluster. The table below summarizes other differences:

<table width="500px">
    <tr>
        <td style="text-align:center;"></td>
        <td style="text-align:center;"><b>K-means</b></td>
        <td style="text-align:center;"><b>EM</b></td>
    </tr>
    <tr>
        <td style="text-align:center;"><b>Cluster representation</b></td>
        <td style="text-align:center;">Mean</td>
        <td style="text-align:center;">Mean, (co)variance</td>
    </tr>
    <tr>
        <td style="text-align:center;"><b>Cluster initialization</b></td>
        <td style="text-align:center;">Randomly select K means</td>
        <td style="text-align:center;">Initialize K Gaussian <br />distributions ($\mu_j$,$\Sigma_j$) and $P(C_j)$</td>
    </tr>
    <tr>
        <td style="text-align:center;"><b>Expectation:</b> <br />
Estimate the cluster of each data</td>
        <td style="text-align:center;">Assign each point to the closest mean </td>
        <td style="text-align:center;">Compute $P(C_j|x_i)$</td>
    </tr>
    <tr>
        <td style="text-align:center;"><b>Maximization:</b> <br /> Re-estimate the cluster parameters</td>
        <td style="text-align:center;">Compute means of current clusters</td>
        <td style="text-align:center;">Compute new ($\mu_j,\Sigma_j$), $P(C_j)$ for each cluster $j$</td>
    </tr>
</table>

If you still curious about EM, you can find [here](https://www.youtube.com/watch?v=REypj2sy_5U) a more detailed explanation.

#### <font color="orange">OpenCV pill</font>

Going back to code, working with EM we have to specify a covariance matrix type using [`em.setCovarianceMatrixType()`](https://docs.opencv.org/3.4/d1/dfb/classcv_1_1ml_1_1EM.html#a8b383c62697eac9a972931674790f6cd). Also, when you applying [`em.trainEM()`](https://docs.opencv.org/3.4/d1/dfb/classcv_1_1ml_1_1EM.html#a5a6a7badbc0c85a8c9fa50a41bf1bcd2) it doesn't return the centroid of the clusters, it is possible to get them calling [`em.getMeans()`](https://docs.opencv.org/3.4/d1/dfb/classcv_1_1ml_1_1EM.html#acec62dd55c06711c81d741c2d96603d1). 

### **<span style="color:green"><b><i>ASSIGNMENT 2: Color quantization with YCrCb color space</i></b></span>**

In the next example, color quantization is realized using the YCrCb color space (more info about such a space in appendix 2) instead of RGB. In this way, color quantization is only applied to the two color bands, neglecting th grayscale one. Let's see how it performs!

**What to do?** Understand and test the following code.

In [ ]:
# Assignment 2
matplotlib.rcParams['figure.figsize'] = (15.0, 15.0)
cv2.setRNGSeed(5)

# Define parameters

n_clusters = 3 # Don't modify this parameter for this exercise

covariance_type = # 0: Spherical covariance matrix. 1: Diagonal covariance matrix. 2: Full covariance matrix
n_iter = 10
epsilon = 0.2

# Create EM empty object


# Set parameters


# Read color image


# Convert to YCrCb


# Take color bands (2 lasts)


# Flatten image


# Apply EM


# Colour resultant labels


# Reshape to original shape


# Merge original first band with quantized color bands


# Cast to unsigned data dype


# Reconvert to RGB


# Show original image


# Show resultant image


### <font color="blue"><b><i>Thinking about it (2)</i></b></font>

Once you understanded the code above, **answer the following questions:**

- Why are the obtained results so good using only 3 clusters?
  
    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- What compression would be better in terms of space in memory, a 16-color compression in a RGB image or a 4-color compression in a YCrCb image?
  
    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>   

### Diving deeper into covariance matrices

There are 3 types of covariance matrices: **spherical covariances**, **diagonal covariances** or **full covariances**: $\\[10pt]$

<img src="./images/ellipses.png" width="700" align="center">$\\[10pt]$ 

###  **<span style="color:green"><b><i>ASSIGNMENT 3: Visualizing clusters from EM</i></b></span>**

Next, you have a code for visualizing the clusters in the YCrCb color space using EM.  

**What to do?** Run the previous example modifying the type of covariance in the EM algorithm and visualize the changes using the following code.

In [ ]:
# Assignment 3
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)

# Get means (2D) and covariance matrices (2x2)
means = np.array(em.getMeans())
covs = np.array(em.getCovs())

# Create figure
fig, ax = plt.subplots()
plt.axis([16, 240, 16, 240])

# Get points contained in each cluster
cluster_1 = np.any(color_bands == np.unique(res,axis=0)[0,:],axis=2)
cluster_2 = np.any(color_bands == np.unique(res,axis=0)[1,:],axis=2)
cluster_3 = np.any(color_bands == np.unique(res,axis=0)[2,:],axis=2)
cluster_1 = image[cluster_1]
cluster_2 = image[cluster_2]
cluster_3 = image[cluster_3]

# Plot them
plt.plot(cluster_1[:,1],cluster_1[:,2],'go')
plt.plot(cluster_2[:,1],cluster_2[:,2],'ro')
plt.plot(cluster_3[:,1],cluster_3[:,2],'bo')

# Plot ellipses representing covariance matrices
PlotEllipse(fig, ax, np.vstack(means[0,:]), covs[0,:,:], 2, color='black')
PlotEllipse(fig, ax, np.vstack(means[1,:]), covs[1,:,:], 2, color='black')
PlotEllipse(fig, ax, np.vstack(means[2,:]), covs[2,:,:], 2, color='black')

fig.canvas.draw()

### <font color="blue"><b><i>Thinking about it (3)</i></b></font>

**Answer the following questions** about how clustering works in EM:

- What are the differences between each type of covariance?
  
    <p style="margin: 4px 0px 6px 5px; color:blue"><i>Your answer here!</i></p>
    
- What type of covariance makes EM equivalent to k-means?
  
    <p style="margin: 4px 0px 0px 5px; color:blue"><i>Your answer here!</i></p>   

### **<span style="color:green"><b><i>ASSIGNMENT 4: Applying EM considering different color spaces</i></b></span>**

It's time to show what have you learned about **EM** and **color spaces**!

**What is your task?** You are asked to **compare color quantization in a RGB color space and in a YCrCb color space**.

For that:
- apply Expectation-Maximization to `malaga.png` using 4 clusters (colors) to both the RGB-space image and the YCrCb-space one,
- and display both results along with the original image.

<font color='blue'>**Expected output:**  </font>

<img src="images/exercise2.png" width="800"/>

In [ ]:
# Assignment 4
matplotlib.rcParams['figure.figsize'] = (15.0, 12.0)
cv2.setRNGSeed(5)

# Define parameters
n_clusters = 4
covariance_type = 2 # 0: covariance matrix spherical. 1: covariance matrix diagonal. 2: covariance matrix generic
n_iter = 10
epsilon = 0.2

# Create EM empty objects


# Set parameters


# Read image


# Convert image to RGB


# Convert image to YCrCb


# Flatten RGB image


# Flatten color bands of YCrCb image


# Apply EM and get centers of clusters


# Colour resultant labels


# Reshape to original shape


# Merge original first band with quantized color bands for YCrCb image


# Cast YCrCb image to unsigned data dype


# Reconvert YCrCb image back to RGB


# Show original image


# Show resultant quantization using RGB color space


# Show resultant quantization using YCrCb color space


## Conclusion

Congratulations for getting this work done! You have learned:

- how k-means clustering works and how to use it,
- how EM algorithm performs and how to employ it,
- how to carry out color quantization and the importance of color spaces in this context, and
- some basics for image compression.

### Extra

You have used YCrCb in this notebook because you were already fimiliar with it. The truth is that, in color quantization matters, [Lab color space](https://en.wikipedia.org/wiki/CIELAB_color_space) is commonly used.  

Surf the internet for information about the Lab color space and then **answer the following questions**:

- How does Lab color space work?
- Why is it typically used for color quantization?

## References

<a name="myfootnote1">[1]</a>: Borenstein, Eran, Eitan Sharon, and Shimon Ullman. [Combining top-down and bottom-up segmentation.](http://www.wisdom.weizmann.ac.il/~vision/courses/2006_2/papers/recog_seg/Borenstein%20combining%20top-down%20and%20bottom-up%20segmentation.pdf). IEEE Conference on Conference on Computer Vision and Pattern Recognition Workshop, 2004.